# Bias, Variance, and Model Validation

![validation gif from giphy](https://media.giphy.com/media/242wLqQerWkxd6GgHB/giphy.gif)

## Learning Objectives

- Define _bias_ and _variance_ in the context of machine learning
- Recognize how bias and variance are related to under- and over-fitting
- Summarize why validation is important
- Describe how a train-test split works
- Apply a train-test split to a dataset using sklearn

Bonus:
- Explain why k-fold cross validation is often more robust than a single train-test split
- Apply k-fold cross validation to a dataset using sklearn

To recap what we've done so far this week, and especially to reiterate what we did yesterday, let's look at some generated data and see what adding polynomial terms does to our model.

In [94]:
# Imports
import pandas as pd
import numpy as np
# Setting random seed for reproducibility
np.random.seed(1000)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# 150 samples from uniform distribution between -2pi and 2pi

x = np.random.uniform(-2*np.pi, 2*np.pi, 150)

# Creating target (y) - so we know the true relationship between x and y
# But - adding some noise (error) with 'np.random'

y = np.sin(x) + np.random.normal(loc=0, scale=0.4, size=len(x))

In [ ]:
# Visualize it
plt.scatter(x, y)

plt.ylabel('$\sin(x)$')
plt.xlabel('x values are randomly chosen from $[-2\pi, 2\pi]$')
plt.show()

In [ ]:
# Fitting a linear model
lr = LinearRegression()
lr.fit(x.reshape(-1, 1), y)

In [ ]:
# Grabbing the predicted values
y_pred = lr.predict(x.reshape(-1, 1))

In [ ]:
# Scoring our model
print(f"R2 Score: {r2_score(y, y_pred)}")

In [ ]:
# Visualize it
plt.scatter(x, y) # original data

plt.plot(x, y_pred, c='red') # predicted values

plt.ylabel('$\sin(x)$ + noise')
plt.xlabel('x values randomly chosen between $-2\pi$ and $2\pi$')
plt.title("Linear Regression")

plt.show()

Is this a good model? Well - of course not. It is what we would call **underfit** - it is not complex enough to accurately capture the pattern and predict the target.

Let's try again, but now with polynomials!

In [ ]:
# For this, we'll need some helper functions
# Shoutout to Andy for sending me these

def create_poly_dataset(x, degree):
    """
    returning dataset with the given polynomial degree
    """
    # Instantiate the PolynomialFeatures object with given 'degree'
    poly = PolynomialFeatures(degree=degree)

    # Now transform data to create higher order features
    new_data = poly.fit_transform(x.reshape(-1, 1))
    return new_data

def fit_linear_model(data, y):
    """
    fitting a linear model and printing model details
    """
    np.set_printoptions(precision=4, suppress=True)

    if data.ndim == 1:
        data = data.reshape(-1, 1)

    lr = LinearRegression(fit_intercept=False)
    lr.fit(data, y)
    print("-"*13)
    print("Coefficients: ", lr.coef_)
    y_pred = lr.predict(data)
    print(f"R-Squared: {lr.score(data, y):.3f}")
    return lr

def plot_predict(x, y, model):
    """
    plotting predictions against true values
    """
    plt.scatter(x, y, label='true')
    x_pred = np.linspace(x.min(), x.max(), 100)
    
    # visualize beyond this x range by uncommenting below:
#     extra = x.ptp() * .2
#     x_pred = np.linspace(x.min() - extra, x.max() + extra, 100)

    plt.plot(x_pred, model.predict(create_poly_dataset(x_pred, len(model.coef_)-1)),
             label='predicted', c='red')

    if len(model.coef_) == 1:
        plt.title(f"{len(model.coef_) - 1} Polynomial Terms \n (no slope)")
    elif (len(model.coef_) - 1) == 1:
        plt.title(f"{len(model.coef_) - 1} Polynomial Term")
    else:
        plt.title(f"{len(model.coef_) - 1} Polynomial Terms")

    plt.legend()
    plt.show()
    return

In [ ]:
# visualizing an assortment of polynomial degrees
# can visualize each sequential polynomial with `range(n)`
for i in [0, 1, 2, 3, 5, 7, 9, 13, 18]:
    xi = create_poly_dataset(x, i)
    plot_predict(x, y, fit_linear_model(xi, y))

So... which one is best?

- 



-------

## The Bias-Variance Trade Off

<img alt="original image from https://rmartinshort.jimdofree.com/2019/02/17/overfitting-bias-variance-and-leaning-curves/" src="images/underfit-goodfit-overfit.png" width=750, height=350>  

Remember - by modeling, we're assuming that there is some relationship between our X variables (the features in our dataset) and our y variable (the target). Thus, there is some underlying '_true_' function that captures the relationship between X and y, which we are trying to find by modeling. Of course, the actual relationship may be quite complex and not wholly represented in our data - our approximation, aka the model we create, is likely only a simplified estimator of whatever our '_true_' function actually would look like.

**Bias**: Error introduced by approximating a real-life problem (which may be extremely complicated) by a much simpler model (because the model is too simple to capture the underlying pattern)

**Variance**: Amount by which our model would change if we estimated it using a different training dataset (because the model is over-learning from the training data)

**Representation:**

<img alt="from https://hsto.org/files/281/108/1e9/2811081e9eda44d08f350be5a9deb564.png" src="images/bias-variance.png/" width=350, height=350>

## How To Minimize Bias and Variance

Good news! There are tried and true methods to reducing both bias and variance in our modeling process. Testing different models, trying models on different slices of data, transforming or engineering features - all of these things have a role to play in creating better, more robust models.

In particular, we've learned so far that we can evaluate the performance of our models, using a scoring metric, which will help us catch if a model is underfit - if it's performing quite poorly, it probably isn't capturing the relationship in our data! 

But what about overfitting?

## Model Validation

Let's say you have a dataframe, with some number of rows of data, and that's all you have available to you. The hope is that you can train a model on this data that can then be used to make predictions about new data that comes in. You want your model to generalize well and work on this incoming data - not too complex from learning all the details/noise from the data, but also not so simple that the model is useless. How do we do that?

<img alt="I Love Lucy shrug gif from Giphy" src="https://media.giphy.com/media/JRhS6WoswF8FxE0g2R/giphy.gif" width=350, height=350>

### Train-Test Split

The idea: don't train your model on ALL of your data, but keep some of it in reserve to test on, in order to simulate how it will work on new/incoming data.

#### Example:

<img alt="original image from https://www.dataquest.io/wp-content/uploads/kaggle_train_test_split.svg plus some added commentary" src="images/traintestsplit_80-20.png" width=850, height=150>  

Note - here, it looks like we're just taking the tail end of the dataset and setting it aside. In practice (most of the time), the split will randomly choose which rows are in the train vs. test sets.

How does this fight against overfitting? By witholding data from the training process, we are testing whether the model actually _generalizes_ well. If it does poorly on the test set, it's a good sign that our model learned too much noise from the train set and is overfit! 

![arrested development gif, found by Andy](https://heavy.com/wp-content/uploads/2013/05/tumblr_mjm9fqhrle1rvnnvyo6_250.gif)

#### Practice:

Let's go back to our Credit data, where we are trying to predict `balance`.

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
df = pd.read_csv('data/Credit.csv',
                 usecols=['Income', 'Limit', 'Rating',
                          'Cards', 'Age', 'Balance'])
df.head()

In [ ]:
df.info()

In [ ]:
# Define our X and y

X = df.drop(columns='Balance')
y = df.Balance

In [ ]:
# Train test split here!
# Set test_size = .33
# Set random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X, y,               # Pass in our X and y
                                                    random_state=42,    # Abritary select a random_state 
                                                    test_size=.33        # Split test size to be 20% of full data.
                                                   )

What did that do?

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
len(X_train + X_test) == len(X)

Now let's put our train/test split into practice:

In [ ]:
# Instantiate a scaler to scale our data
# Let's use Standard Scaler here
ss = StandardScaler()



In [ ]:
# Fit our scaler - ON THE TRAINING DATA!!
# Then transform both train and test 

X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [ ]:
# Instantiate an sklearn linear model

lr = LinearRegression()


In [ ]:
# Fit your model - ON THE TRAINING DATA!!

lr.fit(X_train_scaled, y_train)


In [ ]:
# Grab predictions for train and test set

y_pred_train = lr.predict(X_train_scaled)
y_pred_test = lr.predict(X_test_scaled)

In [ ]:
# How'd we do?

print(f"Train R2 Score: {lr.score(X_train_scaled,y_train)}")
print(f"Test R2 Score: {lr.score(X_test_scaled,y_test)}")

Evaluate!

- Since test score < train score, could be a sign of overfitting.


In [ ]:
sns.heatmap(df.corr().abs())

In [95]:
#Single variable example

X_s = df['Income']
y = df['Balance']

X_s_train, X_s_test, y_s_train, y_s_test = train_test_split(X_s, y,               # Pass in our X and y
                                                    random_state=42,    # Abritary select a random_state 
                                                    test_size=.33        # Split test size to be 20% of full data.
                                                   )

In [96]:
X_s.shape

(400,)

In [97]:
scaler = StandardScaler()

In [102]:
X_s_train = scaler.fit_transform(X_s_train.values,reshape(-1,1))

NameError: name 'reshape' is not defined

In [115]:
X_s_test = scaler.transform(X_s_test.values,reshape(-1,1))

NameError: name 'reshape' is not defined

In [104]:
lr = LinearRegression()

In [105]:
lr.fit(X_s_train,y_s_train)
lr.score(X_s_test,y_s_test)

ValueError: Expected 2D array, got 1D array instead:
array=[ 41.4    22.379  23.012  30.406  69.943  40.885  10.627  75.406  62.413
 107.841  24.088  19.537  93.039  14.711  51.872  28.575  25.383  64.173
  20.996  21.011 163.329  63.931  63.095  28.508 146.183  53.48   52.179
  37.728  20.974  51.345  18.036  27.47   41.868  59.53   28.144  13.676
  23.857  69.165  30.733  49.502  11.808  14.312  19.636 125.48   26.067
  26.532  27.847  64.027  30.007  17.392  62.602  58.929  11.187  27.349
  31.029  38.954  68.462  23.883  27.578  17.055  39.116  55.367  32.856
  27.794  12.414  12.068  24.23   59.855  80.861  68.206  49.927  29.567
  63.534  18.145  11.795  15.045  17.765  30.42   33.437  46.007  58.026
  49.166  24.919  91.362  89.     41.532  20.088  60.449  26.162  76.782
  85.425  54.663  55.056  10.403  50.699  25.974 104.593  57.337  40.442
  69.656  60.579  42.471  27.241  39.609 121.709  53.566  19.349  30.002
  53.308  57.202  16.103  27.229  26.813  19.225  83.948  39.11  

### But Wait... There's More!

Let's change something and see what happens:

In [ ]:
for n in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.33, 
                                                        random_state=n) # <--
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    lr = LinearRegression()
    lr.fit(X_train_scaled, y_train)
    
    y_pred_train = lr.predict(X_train_scaled)
    y_pred_test = lr.predict(X_test_scaled)
    
    print(f"Random Seed: {n}")
    print(f"Train R2 Score: {r2_score(y_train, y_pred_train)}")
    print(f"Test R2 Score: {r2_score(y_test, y_pred_test)}")
    print("-----")

What's happening here? All we're doing is changing our `random_seed` - why is that having such an impact on our model's scores? Some models appear overfit, some don't - and for some, the test score is **better** than our train score!

### K-Fold Cross-Validation

Sometimes, random chance means your training data isn't representative, or includes wacky data like all of our outliers. So, why do just one train-test split when you can do `k` number of them!

![cross validation image from kaggle: https://www.kaggle.com/alexisbcook/cross-validation](images/cross-validation.png)

The good news is, we'll never actually have to do this by hand - `sklearn` will handle it for us!

Documentation: https://scikit-learn.org/stable/modules/cross_validation.html

In [109]:
# Scale our data
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [110]:
# Instantiate a fresh linear regression model
lr = LinearRegression()

In [117]:
# Let's use cross_val_score
# Set cv = 5

scores = cross_val_score(lr,X_scaled,y,cv=5,scoring='r2')

In [118]:
# Look at the test scores across our folds
scores

array([0.8603, 0.8555, 0.8882, 0.8733, 0.8846])

In [119]:
# Print it nicely
print(f"Scores: {scores.mean():.3f} +/- {scores.std():.3f}")

Scores: 0.872 +/- 0.013


#### What the score means above:
R^2 of the test data

Why show the standard deviation of scores here? I want some measure of the variance among my scores, so I can tell how different my scores were based on different breakdowns of the training data.

If I made a change to my model and the average of my cross-validated scores stayed about the same, but the variance among those scores decreased, that's a better, more generalizable model than before!

### Additional Resources:

- [Great bias/variance infographic](https://elitedatascience.com/bias-variance-tradeoff) from Elite Data Science